## 1. Setup

Begin by importing all of the packages used in this notebook. If your kernel uses an environment defined following the guidance on the [tutorials] page, then the imports will be successful.

[tutorials]: https://oceancolor.gsfc.nasa.gov/resources/docs/tutorials

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib

In [2]:
dataset = np.load('../LabelData/dataset.npy')

## Mean and Standard deviation
Compute the mean and stddev of pixels for each of those 19 channels

In [3]:
# Step 1: Extract all image arrays
images = np.array([item[1] for item in dataset])  # shape: (n, 10, 10, 19)
print("images.shape:", images.shape)

# Step 2: Reshape to (n * 10 * 10, 19)
pixels = images.reshape(-1, 19)
print("pixels.shape:", pixels.shape)
print()

# Step 3: Compute mean and std across all valid (non-NaN) pixels for each channel
channel_means = np.nanmean(pixels, axis=0)
channel_stds = np.nanstd(pixels, axis=0)

print("Channel Means (ignoring NaNs):", channel_means)
print("Channel STDs (ignoring NaNs):", channel_stds)

images.shape: (940, 10, 10, 19)
pixels.shape: (94000, 19)

Channel Means (ignoring NaNs): [0.00018529 0.00197223 0.00315226 0.00513789 0.00603405 0.00401628
 0.00471744 0.00849927 0.009139   0.00896765 0.00713521 0.00414049
 0.0049448  0.00285523 0.00410934 0.00269009 0.00399925 0.00281253
 0.00211597]
Channel STDs (ignoring NaNs): [0.00519959 0.00396854 0.00399149 0.00340319 0.00370621 0.00541371
 0.00554312 0.00466338 0.00491752 0.00480148 0.00595262 0.00486634
 0.0040856  0.00444033 0.00357955 0.00435695 0.00340758 0.00437636
 0.00397049]


## EDA
Let's inspect the classes and their distribution

In [4]:
targets = np.array([item[0] for item in dataset])
print("targets.shape:", targets.shape)
unique_vals, counts = np.unique(targets, return_counts=True)
print("Unique values:", unique_vals)
print("Counts:", counts)

targets.shape: (940,)
Unique values: [0 1 2 3 4]
Counts: [408 234 137  93  68]


## Train/Validation split

In [5]:
import torch
from torch.utils.data import random_split

# Example: 80% train, 10% val, 10% test
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # handles rounding

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(63)  # for reproducibility
)

print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))

Train size: 752
Val size: 94
Test size: 94


In [6]:
import torch
from torch.utils.data import Dataset

class MultiChannelDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.data = data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, image = self.data[idx]

        # Convert image to torch tensor (should already be in C x H x W format)
        image = torch.tensor(image, dtype=torch.float32)

        # Transpose the image from (H, W, C) to (C, H, W)
        image = image.permute(2, 0, 1) # Original shape (10, 10, 19) -> Permuted shape (19, 10, 10)

        image = torch.nan_to_num(image, nan=0.0)

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label


In [7]:
from torchvision import transforms

normalize = transforms.Normalize(mean=channel_means, std=channel_stds)

# Pass this to your dataset
train_ds = MultiChannelDataset(train_dataset, transform=normalize)
val_ds = MultiChannelDataset(val_dataset, transform=normalize)
test_ds = MultiChannelDataset(test_dataset, transform=normalize)

for image, label in train_ds:
    print("Label:", label)
    print("Image shape:", image.shape)
    break

# Extract labels
labels = [label for _, label in train_ds]

# Compute weights
class_counts = np.bincount(labels)
class_weights = 1. / class_counts
sample_weights = [class_weights[label] for label in labels]
sample_weights = torch.DoubleTensor(sample_weights)


Label: 2
Image shape: torch.Size([19, 10, 10])


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import numpy as np

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),  # or more for oversampling
    replacement=True
)

In [9]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=4, sampler=sampler)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=4, shuffle=False)

print("Number of batches in train_loader:", len(train_loader))
print("Number of batches in val_loader:", len(val_loader))
print("Number of batches in test_loader:", len(test_loader))

Number of batches in train_loader: 188
Number of batches in val_loader: 24
Number of batches in test_loader: 24


In [10]:
from collections import Counter

for batch_imgs, batch_labels in train_loader:
    print("Batch class distribution:", Counter(batch_labels.tolist()))
    break  # only show first batch


Batch class distribution: Counter({1: 2, 3: 1, 4: 1})


In [11]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 4
num_classes = len(class_counts)
learning_rate = 0.001
num_epochs = 80

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Creating a CNN class
class ConvNeuralNet(nn.Module):
#  Determine what layers and their order in CNN object
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=19, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(64, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    # Progresses data across layers
    def forward(self, x):
        out = self.conv_layer1(x)
#        out = self.conv_layer2(out)
        out = self.max_pool1(out)

        out = self.conv_layer3(out)
#        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        out = out.reshape(out.size(0), -1)

        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

cpu


In [12]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

total_step = len(train_loader)

In [13]:
for (images, labels) in train_loader:
    print(images.shape, labels)
    break

torch.Size([4, 19, 10, 10]) tensor([0, 4, 0, 1], dtype=torch.int32)


In [14]:
torch.autograd.set_detect_anomaly(True)

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
# Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Cast labels to torch.long
        labels = labels.to(torch.long)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/80], Loss: 1.5797
Epoch [2/80], Loss: 1.6456
Epoch [3/80], Loss: 1.6484
Epoch [4/80], Loss: 1.5695
Epoch [5/80], Loss: 1.6769
Epoch [6/80], Loss: 1.5599
Epoch [7/80], Loss: 1.3875
Epoch [8/80], Loss: 1.4186
Epoch [9/80], Loss: 1.0619
Epoch [10/80], Loss: 1.2547
Epoch [11/80], Loss: 1.3362
Epoch [12/80], Loss: 0.8851
Epoch [13/80], Loss: 0.9381
Epoch [14/80], Loss: 1.6843
Epoch [15/80], Loss: 1.0105
Epoch [16/80], Loss: 0.6743
Epoch [17/80], Loss: 1.3848
Epoch [18/80], Loss: 1.2539
Epoch [19/80], Loss: 0.8807
Epoch [20/80], Loss: 1.1220
Epoch [21/80], Loss: 1.0023
Epoch [22/80], Loss: 0.6582
Epoch [23/80], Loss: 0.6458
Epoch [24/80], Loss: 1.6909
Epoch [25/80], Loss: 0.8088
Epoch [26/80], Loss: 1.0710
Epoch [27/80], Loss: 1.7402
Epoch [28/80], Loss: 0.8579
Epoch [29/80], Loss: 0.7316
Epoch [30/80], Loss: 0.8093
Epoch [31/80], Loss: 0.8647
Epoch [32/80], Loss: 0.4441
Epoch [33/80], Loss: 1.1850
Epoch [34/80], Loss: 0.9384
Epoch [35/80], Loss: 0.7988
Epoch [36/80], Loss: 0.5891
E

In [15]:
with torch.no_grad():
    correct = 0
    total = 0

    for which_dataset in [train_loader, val_loader, test_loader]:
      for images, labels in which_dataset:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print('Accuracy of the network on {} batches: {} %'.format(len(which_dataset), 100 * correct / total))

Accuracy of the network on 188 batches: 83.37765957446808 %
Accuracy of the network on 24 batches: 78.95981087470449 %
Accuracy of the network on 24 batches: 74.8936170212766 %


In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from collections import Counter
from torchvision import transforms
from torch.utils.data import random_split, Dataset, DataLoader, WeightedRandomSampler

dataset = np.load('../LabelData/dataset.npy')

# Step 1: Extract all image arrays
images = np.array([item[1] for item in dataset])  # shape: (n, 10, 10, 19)
print("images.shape:", images.shape)

# Step 2: Reshape to (n * 10 * 10, 19)
pixels = images.reshape(-1, 19)
print("pixels.shape:", pixels.shape)
print()

# Step 3: Compute mean and std across all valid (non-NaN) pixels for each channel
channel_means = np.nanmean(pixels, axis=0)
channel_stds = np.nanstd(pixels, axis=0)

print("Channel Means (ignoring NaNs):", channel_means)
print("Channel STDs (ignoring NaNs):", channel_stds)
print()

targets = np.array([item[0] for item in dataset])
print("targets.shape:", targets.shape)
unique_vals, counts = np.unique(targets, return_counts=True)
print("Unique values:", unique_vals)
print("Counts:", counts)
print()

# Example: 80% train, 10% val, 10% test
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # handles rounding

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(63)  # for reproducibility
)

print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))
print("Test size:", len(test_dataset))
print()

class MultiChannelDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.data = data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label, image = self.data[idx]

        # Convert image to torch tensor (should already be in C x H x W format)
        image = torch.tensor(image, dtype=torch.float32)

        # Transpose the image from (H, W, C) to (C, H, W)
        image = image.permute(2, 0, 1) # Original shape (10, 10, 19) -> Permuted shape (19, 10, 10)

        image = torch.nan_to_num(image, nan=0.0)

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label

normalize = transforms.Normalize(mean=channel_means, std=channel_stds)

# Pass this to your dataset
train_ds = MultiChannelDataset(train_dataset, transform=normalize)
val_ds = MultiChannelDataset(val_dataset, transform=normalize)
test_ds = MultiChannelDataset(test_dataset, transform=normalize)

for image, label in train_ds:
    print("Label:", label)
    print("Image shape:", image.shape)
    break
print()

# Extract labels
labels = [label for _, label in train_ds]

# Compute weights
class_counts = np.bincount(labels)
class_weights = 1. / class_counts
sample_weights = [class_weights[label] for label in labels]
sample_weights = torch.DoubleTensor(sample_weights)

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),  # or more for oversampling
    replacement=True
)

train_loader = DataLoader(train_ds, batch_size=4, sampler=sampler)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=4, shuffle=False)

print("Number of batches in train_loader:", len(train_loader))
print("Number of batches in val_loader:", len(val_loader))
print("Number of batches in test_loader:", len(test_loader))
print()

for batch_imgs, batch_labels in train_loader:
    print("Batch class distribution:", Counter(batch_labels.tolist()))
    break  # only show first batch
print()

# Define relevant variables for the ML task
batch_size = 4
num_classes = len(class_counts)
learning_rate = 0.001
num_epochs = 80

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print()

# Creating a CNN class
class ConvNeuralNet(nn.Module):
#  Determine what layers and their order in CNN object
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=19, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(64, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    # Progresses data across layers
    def forward(self, x):
        out = self.conv_layer1(x)
#        out = self.conv_layer2(out)
        out = self.max_pool1(out)

        out = self.conv_layer3(out)
#        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        out = out.reshape(out.size(0), -1)

        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

total_step = len(train_loader)

for (images, labels) in train_loader:
    print(images.shape, labels)
    break
print()

torch.autograd.set_detect_anomaly(True)

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
# Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Cast labels to torch.long
        labels = labels.to(torch.long)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

with torch.no_grad():
    correct = 0
    total = 0

    for which_dataset in [train_loader, val_loader, test_loader]:
      for images, labels in which_dataset:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print('Accuracy of the network on {} batches: {} %'.format(len(which_dataset), 100 * correct / total))

images.shape: (940, 10, 10, 19)
pixels.shape: (94000, 19)

Channel Means (ignoring NaNs): [0.00018529 0.00197223 0.00315226 0.00513789 0.00603405 0.00401628
 0.00471744 0.00849927 0.009139   0.00896765 0.00713521 0.00414049
 0.0049448  0.00285523 0.00410934 0.00269009 0.00399925 0.00281253
 0.00211597]
Channel STDs (ignoring NaNs): [0.00519959 0.00396854 0.00399149 0.00340319 0.00370621 0.00541371
 0.00554312 0.00466338 0.00491752 0.00480148 0.00595262 0.00486634
 0.0040856  0.00444033 0.00357955 0.00435695 0.00340758 0.00437636
 0.00397049]

targets.shape: (940,)
Unique values: [0 1 2 3 4]
Counts: [408 234 137  93  68]

Train size: 752
Val size: 94
Test size: 94

Label: 2
Image shape: torch.Size([19, 10, 10])

Number of batches in train_loader: 188
Number of batches in val_loader: 24
Number of batches in test_loader: 24

Batch class distribution: Counter({0: 1, 4: 1, 3: 1, 1: 1})

cpu

torch.Size([4, 19, 10, 10]) tensor([1, 2, 0, 1], dtype=torch.int32)

Epoch [1/80], Loss: 1.6116
Epoc